In [1]:
import pandas as pd
import numpy as np
import cPickle
import time
import pyprind
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.palettes import Set1_7
output_notebook()
import scipy
import datetime
import decimal

Loading BokehJS ...

In [2]:
src_file = "D:\\course_related_data\\CSEE5690\\A_3\\VM-CPU-Stats-7-Day-2.csv"
store_folder = "D:\\course_related_data\\CSEE5690\\A_3\\"
cpu_data = pd.read_csv(src_file).iloc[::-1]

In [4]:
src_file = "/home/narsi/Downloads/VM-CPU-Stats-7-Day-2.csv"
cpu_data = pd.read_csv(src_file).iloc[::-1]

In [5]:
E_id_list = cpu_data.Entity.unique()
E_select = [25,60,110,225,350]

In [18]:
len(cpu_data[cpu_data['Entity'] == E_id_list[25]]['Value'].tolist())

1442

In [116]:
x = cpu_data[cpu_data['Entity'] == E_id_list[25]]['Timestamp'].values
Time_stamp = []
for i in x:
    y = i.split(' ')
    y = y[-1].split(':')
    y = datetime.timedelta(hours=int(y[0]), minutes=int(y[1]))
    Time_stamp.append(int(decimal.Decimal(y.seconds)/60))

In [10]:
slotx = 156
sloty = 156+288
CPU_FREQ = [cpu_data[cpu_data['Entity'] == E_id_list[25]]['Value'][slotx:sloty].tolist(),
           cpu_data[cpu_data['Entity'] == E_id_list[60]]['Value'][slotx:sloty].tolist(),
           cpu_data[cpu_data['Entity'] == E_id_list[110]]['Value'][slotx:sloty].tolist(),
           cpu_data[cpu_data['Entity'] == E_id_list[225]]['Value'][slotx:sloty].tolist(),
           cpu_data[cpu_data['Entity'] == E_id_list[350]]['Value'][slotx:sloty].tolist()];
VM_ID1 = [CPU_FREQ[0],E_id_list[25]
          +' Mean'+str(np.mean(CPU_FREQ[0]))
         +' Var'+str(np.var(CPU_FREQ[0]))]
VM_ID2 = [CPU_FREQ[1],E_id_list[60]
          +' Mean'+str(np.mean(CPU_FREQ[1]))
         +' Var'+str(np.var(CPU_FREQ[1]))]                            
VM_ID3 = [CPU_FREQ[2],E_id_list[110]
          +' Mean'+str(np.mean(CPU_FREQ[2]))
         +' Var'+str(np.var(CPU_FREQ[2]))]
VM_ID4 = [CPU_FREQ[3],E_id_list[225]
          +' Mean'+str(np.mean(CPU_FREQ[3]))
         +' Var'+str(np.var(CPU_FREQ[3]))]
VM_ID5 = [CPU_FREQ[4],E_id_list[350]
          +' Mean'+str(np.mean(CPU_FREQ[4]))
         +' Var'+str(np.var(CPU_FREQ[4]))]
X_axis = np.arange(0,len(VM_ID1[0]))

In [17]:
p1 = figure(title="Randomly Selected VM's usage on 19th Sep 2016",width=1200, height=500)
p1.xaxis.axis_label = 'time stamp'
p1.yaxis.axis_label = 'CPU Hz'
p1.line(X_axis,VM_ID1[0],legend=VM_ID1[1],color = Set1_7[0], line_width=1,line_dash = 'solid')
p1.line(X_axis,VM_ID2[0],legend=VM_ID2[1],color = Set1_7[1], line_width=1,line_dash = 'dashed')
p1.line(X_axis,VM_ID3[0],legend=VM_ID3[1],color = Set1_7[2], line_width=2,line_dash = 'solid')
p1.line(X_axis,VM_ID4[0],legend=VM_ID4[1],color = Set1_7[3], line_width=2,line_dash = 'dashed')
p1.line(X_axis,VM_ID5[0],legend=VM_ID5[1],color = Set1_7[4], line_width=3,line_dash = 'solid')

p1.legend.location = "top_right"
show(p1)

In [29]:
def update(W,X,V,P):
    
    alpha = 0.5
    
    E = V - P
    
    W += alpha * E * X.T/(np.matmul(X.T,X))
    return W

In [38]:
window = 14
weight_ = np.arange(window,dtype = np.float32)+1
weight_ = weight_/np.sum(weight_)

es_alpha = 0.8
es_trend = 0.5


E_select = [25]

p1 = figure(title="Predicted Value on 09/19/2016 for "+E_id_list[EID],width=900, height=512)
p1.xaxis.axis_label = 'time stamp'
p1.yaxis.axis_label = 'CPU Hz'
for EID in E_select:
    x = cpu_data[cpu_data['Entity'] == E_id_list[EID]]['Timestamp'].values
    Time_stamp = []
    for i in x:
        y = i.split(' ')
        y = y[-2].split(':')
        y = datetime.timedelta(hours=int(y[0]), minutes=int(y[1]))
        Time_stamp.append(int(decimal.Decimal(y.seconds)/60))
    
    V = np.asarray(cpu_data[cpu_data['Entity'] == E_id_list[EID]]['Value'].tolist(),dtype = np.float32)
    p1.line(np.arange(0,288),V[156:156+288],legend='Original',color = Set1_7[0], line_width=1,line_dash = 'solid')
    X = np.empty(len(V)-window)
    
    W = np.random.rand(1,6)/10.0
    A_W = np.random.rand(1,6)/10.0
    # Creating exponential smoothing, exponential smoothing w/ trend
    es = V[0]
    eswt = V[1]
    T1 = 0.0
    for i in range(1,window):
        es = es + (es_alpha) * (V[i-1] - es)
        T1 = T1 + es_trend * (es - V[i-1])
        eswt = es + T1
    counter = 0;
    for i in range(window,len(V)):
        es = es + (es_alpha) * (V[i-1] - es) # Exponential Smooth
        T1 = T1 + es_trend * (es - V[i-1])
        eswt = es + T1  # Exponential Smooth w/ trend
        (ar,br) = scipy.polyfit(np.arange(0,window)+1,V[i-window:i],1)
        lin = scipy.polyval([ar,br],window+1) # Linear Regression
        mavg = np.mean(V[i-window:i]) # Moving Average
        wmavg = np.sum(V[i-window:i] * weight_) # Weighted Moving Average
        
        
        Prev_time = [j for j,x in enumerate(Time_stamp) if x == Time_stamp[i] and j <= i]
        if Prev_time:
            PT = Prev_time[-1]
            if PT == i:
                prev_pridict = np.sum(V[PT-window:PT] * weight_) # Weight moving average
            else:
                prev_pridict = np.mean(V[PT-5:PT+5]) # Weight moving average
        
        
        IN = np.asarray([[es],[eswt],[lin],[mavg],[wmavg],[prev_pridict]])#
        
        X[counter] = np.matmul(W,IN)
        W = update(W,IN,V[i],X[counter])
        
        A_W = np.concatenate((A_W,W),axis=0)
        
        if X[counter] <= 0:
            X[counter] = np.sum(V[i-window:i] * weight_) # Weight moving average
        counter += 1
    p1.line(np.arange(0,288),X[156-window:156+288-window],X,legend='Predicted',color = Set1_7[1], line_width=2,line_dash = 'solid')
    
p1.legend.location = "top_right"
show(p1)

In [39]:
W_names = ['Exp Smooth','Exp Smooth w/ trend','liner','Mov Avg','weight Mov Avg','Previous'];
p1 = figure(title="Update of weights through time",width=900, height=750)
p1.xaxis.axis_label = 'time stamp'
p1.yaxis.axis_label = 'Weights'
for i in range(0,6):
    p1.line(np.arange(1,1439),A_W[:,i], legend=W_names[i],color = Set1_7[i], line_width=2,line_dash = 'solid')
p1.legend.location = "top_left"
show(p1)